# Educarriere

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver

# Fonction pour extraire le texte d'un élément HTML
def extract_text(element, class_name=None, style=None, text_contains=None):
    if element:
        tag = element.find(class_=class_name, style=style, text=text_contains)
        return tag.text.strip() if tag else ""
    else:
        return ""

# Fonction pour nettoyer le texte
def clean_text(text):
    if text is not None:
        cleaned_text = text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')
        return cleaned_text.strip() if cleaned_text else None
    else:
        return None

# Fonction pour extraire la date à partir d'un élément HTML
def extract_date(element, text_contains):
    date_elements = element.find_all('a', class_='text')
    date = next((e.find('span', style='color:#FF0000;').text.strip() for e in date_elements if text_contains in e.text), "")
    return date

# Fonction pour extraire les détails des offres d'emploi
def scrap_job_details(urls):
    # En-tête pour éviter d'être bloqué
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
    options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
    chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
    options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
    options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
    driver = webdriver.Chrome(options=options)

    # Liste pour stocker les détails de chaque emploi
    all_job_details = []

    # Parcourir les liens
    for url in urls:
        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

        offres = soup.find_all('div', class_='box row')

        # Parcourir les offres d'emploi sur la page principale
        for offre in offres:
            # Trouver la balise <h4> dans la structure HTML pour extraire le lien
            offre_link_tag = offre.find('h4')

            # Vérifier si la balise <h4> a été trouvée
            if offre_link_tag:
                # Extraire le lien de l'attribut 'href'
                offre_link = offre_link_tag.find('a')['href']
                all_job_details.append({'Offre_Link': offre_link})

    # Fermer le pilote Selenium à la fin
    driver.quit()

    # Concaténer tous les détails des emplois en un seul DataFrame
    if all_job_details:
        all_job_details_df = pd.DataFrame(all_job_details)
        return all_job_details_df
    else:
        print("Aucun détail d'offre d'emploi trouvé.")
        return None

# Fonction pour récupérer les données des offres d'emploi
def scrape_emploi_ci(url):
    try:
        response = requests.get(url, timeout=200)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {url} : {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    job_description_wrappers = soup.find_all('div', class_='box row')

    data_list = []

    for wrapper in job_description_wrappers:
        h4_tag = wrapper.find('h4')
        poste = extract_text(h4_tag)

        entry_title_tag = wrapper.find('p', class_='entry-title')
        sous_titre = extract_text(entry_title_tag)

        a_text_tag = wrapper.find('a', class_='text')
        code = extract_text(a_text_tag, style='color:#FF0000;')

        date_edition = extract_date(wrapper, "Date d'édition:")
        date_limite = extract_date(wrapper, "Date limite:")

        pays_tag = wrapper.find('a', class_='text')
        pays = pays_tag.find_parent().text.strip().split()[-1] if pays_tag else None

        sous_titre = clean_text(sous_titre)

        data_list.append({
            'Poste': clean_text(poste),
            'Sous_titre': sous_titre,
            'Code': clean_text(code),
            'Date_DEdition': date_edition,
            'Date_limite': date_limite,
            'Pays': clean_text(pays),
            'URL': url  # Ajout de la colonne 'URL'
        })

    df = pd.DataFrame(data_list)
    return df

# Fonction pour ajouter la colonne 'Offre_Link' à un DataFrame existant
def add_offre_link_column(result_df):
    job_details_df = scrap_job_details(list(result_df["URL"]))
    if job_details_df is not None:
        # Join des DataFrames
        result_df = pd.concat([result_df, job_details_df], axis=1)
        return result_df
    else:
        print("Impossible d'ajouter la colonne 'Offre_Link' au DataFrame.")
        return None

# Liste des liens
urls = ["https://emploi.educarriere.ci/nos-offres?page1={}&codes=&mots_cles=&typeemploi1=&niveau1=&anciennete=&typeoffre1=&recruteur=".format(category) for category in range(40)]

# Créer un DataFrame à partir des liens
result_df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)

# Supprimer les lignes dont toutes les variables n'ont pas de données
#result_df = result_df.dropna(how='any').reset_index(drop=True)

# Ajouter la colonne 'Offre_Link'
result_df = add_offre_link_column(result_df)
# Supprimer les lignes avec plus de 80% de valeurs NaN
threshold = int(result_df.shape[1] * 0.8)  # 80% des colonnes
result_df = result_df.dropna(thresh=threshold)
# Réordonner les index
result_df = result_df.reset_index(drop=True)

# Afficher le DataFrame
result_df


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib3
import time

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


# (votre fonction extract_job_information reste inchangée)
def extract_job_information(soup, url):
    try:
        # Extraction des informations de l'offre d'emploi
        poste = soup.select_one('li.list-group-item:-soup-contains("Poste")').strong.next_sibling.strip()
        type_offre = soup.select_one('li.list-group-item:-soup-contains("Type d\'offre")').strong.next_sibling.strip()
        metiers = soup.select_one('li.list-group-item:-soup-contains("Métier(s):")').strong.next_sibling.strip()
        niveaux = soup.select_one('li.list-group-item:-soup-contains("Niveau(x):")').strong.next_sibling.strip()
        experience = soup.select_one('li.list-group-item:-soup-contains("Expérience:")').strong.next_sibling.strip()
        lieu = soup.select_one('li.list-group-item:-soup-contains("Lieu:")').strong.next_sibling.strip()
        
        # Extraction des dates de publication et de limite
        date_publication = soup.find('strong', string='Date de publication:').find_next('span').text.strip()
        date_limite = soup.find('strong', string='Date limite:').find_next('span').text.strip()
        
        # description = soup.select_one('div.text-col.post.small-post.col-md-9.col-xs-12 ul.list-group').text.strip()
        description = soup.select_one('div.entry-content').text.strip()

        return {
            "Poste": [poste],
            "Type d'offre": [type_offre],
            "Métier(s)": [metiers],
            "Niveau(x)": [niveaux],
            "Expérience": [experience],
            "Lieu": [lieu],
            "URL": [url],
            "Date de publication": [date_publication],
            "Date limite": [date_limite],
            "Description": [description]
        }
    except Exception as e:
        print(f"An error occurred while extracting job information for URL {url}: {e}")
        # Retourner un dictionnaire avec l'URL en cas d'erreur
        return {"URL": [url]}

# Liste des URLs à scraper
urls = list(result_df['Offre_Link'])

# Liste pour stocker les DataFrames
dfs = []

# Boucle sur chaque URL
for url in urls:
    try:
        # Envoyer une requête GET au site avec un délai de 120 secondes
        response = requests.get(url, headers=headers, verify=True, timeout=120)

        # Vérifier si la requête a réussi (statut 200)
        if response.status_code == 200:
            # Analyser le contenu de la page avec BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extraire les informations sur l'emploi
            job_info = extract_job_information(soup, url)

            # Créer un DataFrame
            df = pd.DataFrame(job_info)

            # Ajouter le DataFrame à la liste
            dfs.append(df)
        else:
            print(f"Échec de la requête pour l'URL {url}. Statut : {response.status_code}")
            # Ajouter une ligne avec l'URL en cas d'erreur
            dfs.append(pd.DataFrame({"URL": [url]}))

    except requests.exceptions.Timeout:
        print(f"Timeout lors de la requête pour l'URL {url}")
        # Ajouter une ligne avec l'URL en cas d'erreur
        dfs.append(pd.DataFrame({"URL": [url]}))
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite lors de la requête pour l'URL {url}: {e}")
        # Ajouter une ligne avec l'URL en cas d'erreur
        dfs.append(pd.DataFrame({"URL": [url]}))

    # Ajouter un délai de 5 secondes entre les requêtes pour éviter d'être bloqué
    time.sleep(5)

# Concaténer tous les DataFrames en un seul DataFrame
df_Educarriere = pd.concat(dfs, ignore_index=True)
df_Educarriere


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib3
import time

# Désactiver les avertissements liés aux requêtes non sécurisées
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# (votre fonction extract_job_information reste inchangée)
def extract_job_information(soup):
    # Extraction des informations de l'offre d'emploi
    poste = soup.select_one('li.list-group-item:-soup-contains("Poste")').strong.next_sibling.strip()
    type_offre = soup.select_one('li.list-group-item:-soup-contains("Type d\'offre")').strong.next_sibling.strip()
    metiers = soup.select_one('li.list-group-item:-soup-contains("Métier(s):")').strong.next_sibling.strip()
    niveaux = soup.select_one('li.list-group-item:-soup-contains("Niveau(x):")').strong.next_sibling.strip()
    experience = soup.select_one('li.list-group-item:-soup-contains("Expérience:")').strong.next_sibling.strip()
    lieu = soup.select_one('li.list-group-item:-soup-contains("Lieu:")').strong.next_sibling.strip()
    
    # Extraction des dates de publication et de limite
    date_publication = soup.find('strong', string='Date de publication:').find_next('span').text.strip()
    date_limite = soup.find('strong', string='Date limite:').find_next('span').text.strip()
    
   # description = soup.select_one('div.text-col.post.small-post.col-md-9.col-xs-12 ul.list-group').text.strip()
    description = soup.select_one('div.entry-content').text.strip()

    return {
        "Poste": [poste],
        "Type d'offre": [type_offre],
        "Métier(s)": [metiers],
        "Niveau(x)": [niveaux],
        "Expérience": [experience],
        "Lieu": [lieu],
        "Date de publication": [date_publication],
        "Date limite": [date_limite],
        "Description": [description]
    }

# Liste des URLs à scraper
urls = list(all_job_details_df['Offre_Link'])

# Liste pour stocker les DataFrames
dfs = []

# Boucle sur chaque URL
for url in urls:
    try:
        # Envoyer une requête GET au site avec un délai de 120 secondes
        response = requests.get(url, headers=headers, verify=True, timeout=120)

        # Vérifier si la requête a réussi (statut 200)
        if response.status_code == 200:
            # Analyser le contenu de la page avec BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')

            try:
                # Extraire les informations sur l'emploi
                job_info = extract_job_information(soup)
                job_info['URL'] = [url]

                # Créer un DataFrame
                df = pd.DataFrame(job_info)

                # Ajouter le DataFrame à la liste
                dfs.append(df)

            except Exception as e:
                df = pd.DataFrame({"URL": [url]})
                 
                # Ajouter le DataFrame à la liste
                dfs.append(df)
                print(f"An error occurred while extracting job information: {e}")
        else:
            df = pd.DataFrame({"URL": [url]})
            dfs.append(df)
            print(f"Échec de la requête pour l'URL {url}. Statut : {response.status_code}")

    except requests.exceptions.Timeout:
        df = pd.DataFrame({"URL": [url]})
        dfs.append(df)
        print(f"Timeout lors de la requête pour l'URL {url}")
    except requests.exceptions.RequestException as e:
        df = pd.DataFrame({"URL": [url]})
        dfs.append(df)
        print(f"Une erreur s'est produite lors de la requête pour l'URL {url}: {e}")

    # Ajouter un délai de 5 secondes entre les requêtes pour éviter d'être bloqué
    time.sleep(5)

# Concaténer tous les DataFrames en un seul DataFrame
df_Educarriere = pd.concat(dfs, ignore_index=True)
df_Educarriere


# Novojob

In [52]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Liste des liens pour chaque catégorie
categories = [
    "toutes les offres d'emploi",
    "juridique,fiscal,audit,conseil",
    "administrations,moyens généraux",
    "assistanat,secrétariat",
    "metiers banque et assurances",
    "RH,personnel,formation",
    "education,enseignement",
    "direction générale,direction d'unité",
    "vente,televente,assistanat",
    "commercial,technico commercial,service client",
    "responsable commercial,grands comptes",
    "créatio, design",
    "marketing, communication",
    "journalisme,médias,traduction",
    "informatique,systèmes d'information,internet",
    "télécommunication,réseaux",
    "chantier,métiers BTP,architecture",
    "ingénierie,etudes,projet,R&D",
    "logistique,achat,stock,transport",
    "production,méthode,industrie",
    "maintenance,entretien",
    "Qualité,sécurité,Environnement",
    "Santé,Médical,Pharmacie",
    "Hotellerie,Tourisme,Restauration, Loisirs",
    "Ouvriers qualifiés, Chauffeur",
    "autre",
    "Métiers de l'agriculture"
]

base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
niveau_list = []
experience_list = []
lien_list = []
all_job_lien = []

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_='row-fluid job-details pointer')

    for offre in offres:
        offre_link_tag = offre.find('a')
        if offre_link_tag:
            offre_link = offre_link_tag['href']
            all_job_lien.append(offre_link)

        entreprise_tag = offre.find('h6', class_='ellipsis')
        entreprise = entreprise_tag.get_text().strip() if entreprise_tag else None

        intitule_tag = offre.find('h2', class_='ellipsis row-fluid')
        intitule = intitule_tag.get_text().strip() if intitule_tag else None

        bloc_bottom = offre.find_next('div', class_='bloc-bottom')

        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None

        niveau_info = bloc_bottom.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        match = re.match(r'(.+) \((.+)\)', niveau_text)
        niveau_col, experience_col = match.groups() if match else (None, None)

        intitules_list.append(intitule)
        entreprises_list.append(entreprise)
        pays_list.append(pays)
        dates_list.append(date)
        niveau_list.append(niveau_col)
        experience_list.append(experience_col)
        url_list.append(category_link)

# Création du DataFrame
df_offers = pd.DataFrame({
    'Intitule': intitules_list,
    'Entreprise': entreprises_list,
    'Pays': pays_list,
    'Date': dates_list,
    'Niveau': niveau_list,
    'Experience_lettre': experience_list,
    'url': url_list,
    'Offre_Link': all_job_lien
})

df_offers


,Intitule,Entreprise,Pays,Date,Niveau,Experience_lettre,url,Offre_Link
0,Livreurs Moto,Entreprise anonyme,Côte d'ivoire,04 Janvier,Confirmé / Expérimenté,Sans expérience,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
1,Pompiste H/F,None,"Abidjan, Côte d'ivoire",27 Décembre 2023,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
2,Caissier H/F,Coris consulting,Côte d'ivoire,27 Décembre 2023,Débutant / Junior,Sans expérience,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
3,Ouvrier de rayon,Coris consulting,Côte d'ivoire,27 Décembre 2023,Débutant / Junior,Sans expérience,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
4,Commercial Terrain,KOMIAN AI,Côte d'ivoire,02 Février,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
...,...,...,...,...,...,...,...,...
706,Responsable Clientèle Patrimoniale (H/F),Société Ivoirienne de Banque (SIB),"Abidjan, Côte d'ivoire",05 Février,Confirmé / Expérimenté,3 à 5 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
707,Technicien Génie Civil Option Bâtiment,SOCOPI,Côte d'ivoire,17 Janvier,Débutant / Junior,1 à 2 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
708,Chargé de Support Système Windows et Virtualis...,Ascens,"Abidjan, Côte d'ivoire",24 Novembre 2023,Confirmé / Expérimenté,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
709,Chargé.e de Dossiers Achats et Contrats (Assis...,Giz Côte d'Ivoire,"Korhogo, Côte d'ivoire",01 Février,Débutant / Junior,1 à 2 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...


In [53]:
import requests
import pandas as pd

# Example usage for multiple job URLs
job_urls = list(df_offers['Offre_Link'])

# Extract details for each job URL
all_job_details = []
for url in job_urls:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')

    job_details = {}
    
    # Ajouter le lien
    job_details["Offre_Link"]=url
    # Extracting job details
    details_section = soup.find('ul', class_='text-small')
    if details_section:
        for li in details_section.find_all('li', class_='row-fluid'):
            key = li.find('span', class_='span4').text.strip()
            value = li.find('span', class_='span8').text.strip()
            job_details[key] = value

    # Extracting the provided text
    description_section = soup.find('div', class_='spaced details-description')
    if description_section:
        provided_text = description_section.text.strip()
        job_details['Provided Text'] = provided_text

    all_job_details.append(job_details)

# Create a DataFrame
df_Novojob = pd.DataFrame(all_job_details)
df_Novojob

ReadTimeout: HTTPSConnectionPool(host='www.novojob.com', port=443): Read timed out. (read timeout=None)

# Emploi.ci

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_text(element, tag_name=None):
    tag = element.find(tag_name)
    return tag.text.strip() if tag else ""

def clean_text(text):
    return text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')

def scrape_emploi_ci(url):
    try:
        response = requests.get(url, timeout=500)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {url} : {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    job_description_wrappers = soup.find_all('div', class_='job-description-wrapper')

    data_list = []

    for wrapper in job_description_wrappers:
        h5_tag = wrapper.find('h5')
        poste = extract_text(h5_tag, 'a')

        job_recruiter_tag = wrapper.find('p', class_='job-recruiter')
        date_and_company = job_recruiter_tag.text.strip().split('|')
        date = date_and_company[0].strip() if date_and_company else ""
        entreprise = extract_text(job_recruiter_tag, 'a')

        description_tag = wrapper.find('div', class_='search-description')
        description = clean_text(description_tag.text.strip()) if description_tag else ""

        region_tag = wrapper.find('p', text='Région de :')
        region = extract_text(region_tag) if region_tag else ""

        data_list.append({
            'Poste': poste,
            'Entreprise': entreprise,
            'Date': date,
            'Description': description,
            'Région': region,
            "URL":url

        })

    df = pd.DataFrame(data_list)
    return df


# Fonction pour extraire les détails des offres d'emploi
def scrap_job_details(urls):
    # En-tête pour éviter d'être bloqué
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
    options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
    chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
    options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
    options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
    driver = webdriver.Chrome(options=options)

    # Liste pour stocker les détails de chaque emploi
    all_job_details = []

    # Parcourir les liens
    for url in urls:
        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

        offres = soup.find_all('div', class_="job-description-wrapper")

        # Parcourir les offres d'emploi sur la page principale
        for offre in offres:
            # Trouver la balise <h4> dans la structure HTML pour extraire le lien
            offre_link_tag = offre.find('h5')

            # Vérifier si la balise <h4> a été trouvée
            if offre_link_tag:
                # Extraire le lien de l'attribut 'href'
                offre_link = offre_link_tag.find('a')['href']
                all_job_details.append({'Offre_Link': "https://www.emploi.ci"+offre_link})

    # Fermer le pilote Selenium à la fin
    driver.quit()

    # Concaténer tous les détails des emplois en un seul DataFrame
    if all_job_details:
        all_job_details_df = pd.DataFrame(all_job_details)
        return all_job_details_df
    else:
        print("Aucun détail d'offre d'emploi trouvé.")
        return None
    
    
    
# Fonction pour ajouter la colonne 'Offre_Link' à un DataFrame existant
def add_offre_link_column(emploi_df):
    job_details_df = scrap_job_details(list(emploi_df["URL"]))
    if job_details_df is not None:
        # Join des DataFrames
        result_df = pd.concat([emploi_df, job_details_df], axis=1)
        return emploi_df
    else:
        print("Impossible d'ajouter la colonne 'Offre_Link' au DataFrame.")
        return None

# Liste des liens
categories = ["31", "1127", "29", "37", "1115", "30", "1115", "32", "33", "34", "35", "36", "37", "39", "38", "40", "525", "41", "28"]
#categories=["31"]
# Liste d'URLs générées
urls = ["https://www.emploi.ci/recherche-jobs-cote-ivoire/?f%5B0%5D=im_field_offre_metiers%3A{}".format(category) for category in categories]

# Créer un DataFrame à partir des liens
emploi_df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)

# Supprimer les lignes dont toutes les variables n'ont pas de données
#result_df = result_df.dropna(how='any').reset_index(drop=True)

# Ajouter la colonne 'Offre_Link'
emploi_df = add_offre_link_column(emploi_df)
# Supprimer les lignes avec plus de 80% de valeurs NaN
threshold = int(emploi_df.shape[1] * 0.8)  # 80% des colonnes
emploi_df = emploi_df.dropna(thresh=threshold)
emploi_df=emploi_df.reset_index(drop=True)
# Afficher le DataFrame
emploi_df